# 3.1 Student Mask Detector

#### Standard Imports

In [1]:
import classroom_mask_detector_essestials as cmde  # All the Custom classes, functions and Constants for this project

In [2]:
import numpy as np
import cv2
import face_recognition

from keras.models import load_model
import os
from time import time

#### Email Credentials

In [3]:
MY_EMAIL = None
PASSWORD = None
AUTHORITY_EMAIL = None

##### The face detector and the mask detector

In [4]:
face_detector = cv2.CascadeClassifier('./Data/haarcascade_frontalface_default.xml')

In [5]:
mask_detector = load_model('./Data/mask_detection_model.model')

#### Functions to simplify the code

In [6]:
def predict_class_and_probability(image):
    prediction = mask_detector.predict(image)
    class_ = np.argmax(prediction)  # getting the index of the hightes prediction
    probability = np.max(prediction)
    return class_, probability

In [7]:
def resize_preprocess_reshape_image(image):
    image = cv2.resize(image, (cmde.IMAGE_SIZE, cmde.IMAGE_SIZE)) # Resize the Image to the dimensions
    image = cmde.preprocess_image(image)  
    image = image.reshape(1, cmde.IMAGE_SIZE, cmde.IMAGE_SIZE, 1)
    return image

#### The main loop

In [8]:
webcam = cv2.VideoCapture(0)

known_faces = cmde.KnownFaceEncodings(new_image_added=True)
# make sure the lists of student is None
cmde.IdentifiedFace.flush()
cmde.UnidentifiedFace.flush()

time_run = 0
t1 = time() # get the initial time

while cmde.RUN_FOR > time_run:
    sucess, frame = webcam.read()
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Converting the image to grey for better face recognition
    faces = face_detector.detectMultiScale(frame_gray, 1.3, 5)  # Detect for faces in the current frame

    for x, y, w, h in faces: # for x_cor, y_cor, width, height of every face
        crop_img = frame[y: y+h, x: x+h]  # crop the frame just to the face using list slicing
        
        img = resize_preprocess_reshape_image(crop_img) 

        # Pridict for the face (if it is wearing mask or not)
        class_, probability = predict_class_and_probability(img)

        if probability >= cmde.THRESHOLD and class_ == cmde.WITHOUT_MASK:
            
            face = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
            encoded_face = face_recognition.face_encodings(face)
            
            if len(encoded_face) >= 1:
                # get the matches and the distance to identify the name of the person who is not wearing a mask 
                matches = face_recognition.compare_faces(known_faces.encodings, encoded_face)
                distance = face_recognition.face_distance(known_faces.encodings, encoded_face)
                idx = np.argmin(distance)

                if matches[idx]:  # if the comparision is true then add it to the identified faces
                    name = known_faces.names[idx]
                    cmde.IdentifiedFace(name, crop_img)  # Add the identified face
                    
                else:
                    cmde.UnidentifiedFace(crop_img, encoded_face)  # Add the unidentified face
    
    cv2.imshow("Webcam", frame)
    key = cv2.waitKey(1) # to force quit
    if key == ord('q') or key == ord('Q'):
        break
        
    t2 = time()
    time_run = t2 - t1 # update the time
        
webcam.release()
cv2.destroyAllWindows()
print('Sending the mails if needed...')

# Report to the authorities
cmde.IdentifiedFace.report_to_authority(MY_EMAIL, PASSWORD, AUTHORITY_EMAIL)
cmde.UnidentifiedFace.report_to_authority(MY_EMAIL, PASSWORD, AUTHORITY_EMAIL)
print('Done!')

C:\Users\Family\anaconda3\lib\site-packages\PIL\Image.py:951: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Sending the mails if needed...
[IdentifiedFace] No students to report
 Credentials are missing please fill them out! 


TypeError: argument of type 'NoneType' is not iterable

### The result email is 
![demo email](./Assets/mail_by_final_project.png)
# Thank You 